# Analyse Musique

### Importation des bibliothèques et définition de variables générales

In [53]:
import vamp
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from math import ceil


notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
accords = [f"{note} Major" for note in notes] + [f"{note} Minor" for note in notes] + ["N"] # Tous les accords Majeurs et Mineurs + "N" = pas d'accord qui joue
std_notation = { # Notation à utiliser pendant l'entrainement
    "Db" : "C#",
    "Eb" : "D#",
    "Gb" : "F#",
    "Ab" : "G#",
    "Bb" : "A#",
    "Fb" : "E",
    "Cb" : "B"
}

gamme_maj = [1, 0, 0, 1, 1, 0, "dim"] # 1 signifie un accord Majeur, un 0 un accord Mineur, "dim" un accord diminué (sera ignoré car pas un accord dans la base de données)

In [82]:
print(accords)

['C Major', 'C# Major', 'D Major', 'D# Major', 'E Major', 'F Major', 'F# Major', 'G Major', 'G# Major', 'A Major', 'A# Major', 'B Major', 'C Minor', 'C# Minor', 'D Minor', 'D# Minor', 'E Minor', 'F Minor', 'F# Minor', 'G Minor', 'G# Minor', 'A Minor', 'A# Minor', 'B Minor', 'N']


### Application de l'algorithme de Krumhansl-Schmuckler pour déterminer la clé d'une chanson

In [89]:
# SOURCE D'AIDE: https://github.com/jackmcarthur/musical-key-finder/

profile_maj = [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
profile_min = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]
def trouve_cle(audio):
    assert librosa.get_duration(y=audio) > 0, "Le fichier peut être corrompu." # Un bug qui se produit parfois, car un fichier musique a une longueur 0
    chromagram = librosa.feature.chroma_cqt(y=audio)
    presence_notes = []
    for i in chromagram:
        presence_notes.append(np.sum(i))
    correlation_maj, correlation_min = [], []
    for i in range(12):
        cle_test = [presence_notes[(i+j)%12] for j in range(12)]
        correlation_maj.append(np.corrcoef(profile_maj, cle_test)[0][1])
        correlation_min.append(np.corrcoef(profile_min, cle_test)[0][1])
    correlation = correlation_maj + correlation_min
    cle = []
    for i in range(2):
        base_cle = correlation.index(max(correlation))
        cle.append(base_cle)
        if base_cle > 11:
            base_cle_relatif = (base_cle + 3)%12
            cle.append(base_cle_relatif)
            correlation[base_cle_relatif] = -50
        else:
            base_cle_relatif = (base_cle + 9)%12+12
            cle.append(base_cle_relatif)
            correlation[base_cle_relatif] = -50
        cle.append(correlation[base_cle])
        correlation[base_cle] = -50
    return cle

### Création des fichiers de données pour le model des K Plus Proches Voisins pour la détermination des accords. On utilise cette base de données https://ddmal.music.mcgill.ca/research/The_McGill_Billboard_Project_(Chord_Analysis_Dataset)/

In [ ]:
def get_data():
    X, Y = [], []
    for i in os.listdir("/Chord recognition/mcgill-chords"):
        if i != ".DS_Store":
            print(i)
            with open("/Chord recognition/mcgill-chords/" + i + "/majmin.lab") as chords:
                chords = [j.strip("\n").split("\t") for j in chords.readlines()[:-1:]]
                
                # Fusionner les listes ayant les même accords
                merged_chords = []
                prev_chord = None
                for p in range(len(chords)-1):
                    if chords[p][2] != prev_chord:
                        k = 1
                        try:
                            while chords[p][2] == chords[p+k][2]:
                                k += 1
                        except IndexError:
                            pass
                        merged_chords.append((chords[p][0], chords[p+k-1][1], chords[p][2]))
                        prev_chord = chords[p][2]
                    
                with open("/Chord recognition/mcgill-chroma/" + i + "/bothchroma.csv") as chroma:
                    chroma = [j.strip("\n").strip('"/tmp/audio.wav"').split(",")[1:] for j in chroma.readlines()]
                    for n in merged_chords:
                        limites_temps = (float(n[0]), float(n[1]))
                        limites_indices = [-1, -1]
                        k = 0
                        while float(chroma[k][0]) < limites_temps[0]:
                            k += 1
                        if k != 0:
                            k += 1
                        limites_indices[0] = k
                        
                        k = 0
                        while k < len(chroma) and float(chroma[k][0]) < limites_temps[1]:
                            k += 1
                        if k == len(chroma):
                            k -= 1
                        limites_indices[1] = k
                        
                        segment_chromas = np.array([np.array(chroma[l][1:]).astype(np.float64) for l in range(limites_indices[0], limites_indices[1]+1)])
                        average_chroma = np.mean(segment_chromas, axis=0)
                        
                        mode = None
                        mode_accord = None
                        if n[2][0] == "X":
                            continue
                        elif len(n[2]) == 1:
                            note = "N"
                        else:
                            note, mode = n[2].split(":")
                        
                        if note not in notes and note != 'N':
                            note = std_notation[note]
                        if mode == "maj":
                            label = accords.index(note + " Major")
                        elif mode == "min":
                            label = accords.index(note + " Minor")
                        else:
                            label = accords.index(note)
                        if average_chroma.shape != (24,):
                            print("ERREUR")
                            continue
                        X.append(average_chroma)
                        Y.append(label)
    np.save("x_train", np.array(X))
    np.save("y_train", np.array(Y))
get_data()

### Fonction pour obtenir le model

In [56]:
def LoadModel(k_voisins=1):
    X = np.load("x_train.npy")
    Y = np.load("y_train.npy")
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    knn = KNeighborsClassifier(n_neighbors=k_voisins)
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return knn, accuracy

### On met k_voisins à 1 par défaut car, selon le script suivant, cela nous donne la plus grande précision

In [ ]:
def trouveMeilleurK():
    precisions = []
    for i in range(1, 31):
        precisions.append(LoadModel(i)[1])
    print(f"{precisions.index(max(precisions)) + 1} --> {max(precisions)}")

trouveMeilleurK()

### Création de la fonction pour déterminer les accords

In [95]:
def trouve_accords(audio, sr, knn, diatonique=False, cle_maj=None):
    tempo, beats = librosa.beat.beat_track(y=audio, sr=sr)
    beats_time = librosa.frames_to_time(beats, sr=sr)
    beats_samples = librosa.frames_to_samples(beats)
    chromas = []
    temps_accord = []
    hop = ceil(tempo/40)
    for i in range(0, len(beats)-hop, hop):
        y = audio[beats_samples[i]:beats_samples[i+hop]+1]
        temps_accord.append([float(beats_time[i]), float(beats_time[i+hop])])
        chroma = vamp.collect(y, sr, "nnls-chroma:nnls-chroma", output="bothchroma", parameters={"rollon":0.01})["matrix"][1]
        average_chroma = np.mean(chroma, axis=0)
        chromas.append(average_chroma)
    index_accords = knn.predict(chromas)
    for i in range(len(index_accords)):
        temps_accord[i].append(int(index_accords[i]))
    
    if diatonique and cle_maj != None:
        index_base = cle_maj
        notes_cle = [index_base, (index_base + 2)%12, (index_base + 4)%12, (index_base + 5)%12,
                     (index_base + 7)%12, (index_base + 9)%12, (index_base + 11)%12]
        for i in range(len(temps_accord)):
            if (temps_accord[i][2]%12 not in notes_cle) or (gamme_maj[notes_cle.index(temps_accord[i][2]%12)] == "dim") or (temps_accord[i][2] > 11 and gamme_maj[notes_cle.index(temps_accord[i][2]%12)] == 1) or (temps_accord[i][2] < 12 and gamme_maj[notes_cle.index(temps_accord[i][2]%12)] == 0):
                if temps_accord[0] == temps_accord[i]:
                    temps_accord.pop(0)
                else:
                    temps_accord[i][2] = temps_accord[i-1][2]
                
    return temps_accord